In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('it',extract_format=wikipediaapi.ExtractFormat.WIKI)
#This is quick fix to make the word embeddings parametric
#The problem is that the Prelearn dataset is Italian and it relies on Wikipedia APIs to get the English equivalent
#Therefore the embeddings need those APIs, but they're not needed by the other datasets
#I could translated the dataset here but I tried and it takes waaaaay too much
is_Prelearn = True

#dataset init

#file consisting of pairs of target and prerequisite concepts (A, B) labelled as follows:
#1 if B is a prerequisite of A;
#0 in all other cases.
df = pd.read_csv('/content/drive/My Drive/train.csv')
train = df.sample(frac=0.8,random_state=200) 
validation = df.drop(train.index)

#The Wikipedia page of each concept found in the previous file.
#Each Wikipedia page is introduced by a `<doc>` element (with *id* and *url*) 
#containing the title and the text of the corresponding page.

#THE PAGES HERE ARE IN ITALIAN
#Since we want the English equivalent, we will only get the page title, and then use the Wikipedia api to get its translation

tree = ET.parse('/content/drive/My Drive/dataset.xml')
pages = tree.getroot()

In [ ]:
#create dictionary of tokenized documents
punct = string.punctuation + '«``»' + "''"
en_stop_words = nltk.corpus.stopwords.words('english')

doc_dict = {}

#we get the english wikipedia pages

for content in pages.iter('doc'):
  #Italian title
  title = content.find('title').text
  #Eng page
  p_wiki = wiki_wiki.page(title).langlinks['en']
  #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
  document = (p_wiki.title + " "  + p_wiki.text).replace("'"," ")
  document = document.replace("\\"," ")
  document = document.replace("displaystyle"," ")
  #tokenized
  tokenized = nltk.tokenize.word_tokenize(document, "english")
  #no punctuation and lowercase
  no_punct = [x.lower() for x in tokenized if x not in punct]
  #remove stop words
  no_stop_words = [x for x in no_punct if x not in en_stop_words]

    #add tokenized document
  doc_dict[content.find('title').text] = no_stop_words